<a href="https://colab.research.google.com/github/purvanshiporwal0609/PRODIGY_GA_1/blob/main/PRODIGY_GA_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sample_motivation_dataset.txt to sample_motivation_dataset (2).txt


In [ ]:
from datasets import Dataset

file_path = list(uploaded.keys())[0]
with open(file_path, 'r') as f:
    text_data = f.readlines()

dataset = Dataset.from_dict({"text": text_data})

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


In [ ]:
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()  # Add labels for language modeling
    return tokenized_inputs

tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

# Split the tokenized dataset using the datasets library's train_test_split
train_test_split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = train_test_split_dataset['train']
test_dataset = train_test_split_dataset['test']

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=200,
    save_total_limit=2,
    logging_steps=10,
)

# Initialize the Data Collator for Language Modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False # mlm=False for causal language modeling
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, # It's good practice to include an evaluation dataset
    data_collator=data_collator # Add the data collator here
)

In [ ]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.341800
20,1.694900


TrainOutput(global_step=27, training_loss=1.9488073984781902, metrics={'train_runtime': 239.6153, 'train_samples_per_second': 0.225, 'train_steps_per_second': 0.113, 'total_flos': 3527442432000.0, 'train_loss': 1.9488073984781902, 'epoch': 3.0})

In [ ]:
trainer.save_model("finetuned-gpt2-motivation")


In [ ]:
from transformers import pipeline

text_generator = pipeline("text-generation", model="finetuned-gpt2-motivation", tokenizer=tokenizer)

prompt = "Never lose hope as"
output = text_generator(prompt, max_length=50, num_return_sequences=1)

print(output[0]["generated_text"])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Never lose hope as you strive to live, but you must also be able to do everything in your power to love and grow.

If you love something, you do it.

If you're struggling with a disability, but you never lose hope as you seek to do amazing things that will make you stronger.

If you're struggling with an addiction that you've never experienced, but you're strong and determined to do what's right, you can do it all.

If you're struggling with a mental illness, but you're strong and determined to do what's right, you can do it all.

If you feel hopeless and need help, but you're able to do it all, you can reach out to others, and do much more than just try to live your life.

If you're struggling with a financial problem, but you're strong and determined to do what's right, you can do it all.

If you're struggling with an addiction that you've never experienced, but you're determined to do what's right, you can do it all.

If you're struggling with a medical problem, but you're strong an